In [ ]:
!pip install audio_classification_features

  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp36-none-any.whl size=5887 sha256=f24cac159f23f9364986b7f1467b22746a2fd82a6460c40aca0ba669b76f726d
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [ ]:
!git clone "https://github.com/Jakobovski/free-spoken-digit-dataset.git"

Cloning into 'free-spoken-digit-dataset'...
remote: Enumerating objects: 506, done.
remote: Counting objects: 100% (506/506), done.
remote: Compressing objects: 100% (506/506), done.
remote: Total 4178 (delta 0), reused 501 (delta 0), pack-reused 3672
Receiving objects: 100% (4178/4178), 30.44 MiB | 28.03 MiB/s, done.
Resolving deltas: 100% (77/77), done.


In [ ]:
!mkdir dataset

In [ ]:
!mv /content/free-spoken-digit-dataset/recordings/* /content/dataset/

In [ ]:
!rm -rf /content/free-spoken-digit-dataset/

In [ ]:
!mkdir cl_dataset

In [ ]:
!cd cl_dataset/ && mkdir 0 && mkdir 1 && mkdir 2 && mkdir 3 && mkdir 4 && mkdir 5 && mkdir 6 && mkdir 7 && mkdir 8 && mkdir 9

In [ ]:
!mkdir test

In [ ]:
import shutil
import os

In [ ]:
path='/content/dataset'
files=[]
i=0
train=0
test=0
for r,d,f in os.walk(path):
  for file in f:
    if i==48:
      i=0
    dir_to_go=str(file[0])
    file_name=str(file)
    if i<=10:
      shutil.move(path+'/'+file_name,"/content/test/"+file_name)
      test+=1
    else:
      shutil.move(path+'/'+file_name,"/content/cl_dataset/"+dir_to_go+'/'+file_name)
      train+=1
    i+=1
print("Train: {}".format(train))
print("Test: {}".format(test))

7_yweweler_34.wav
4_lucas_45.wav
1_yweweler_18.wav
6_yweweler_28.wav
7_nicolas_14.wav
0_jackson_27.wav
8_yweweler_44.wav
2_lucas_10.wav
2_jackson_12.wav
6_jackson_35.wav
5_theo_40.wav
5_george_15.wav
3_lucas_30.wav
1_george_29.wav
2_george_47.wav
4_theo_8.wav
5_yweweler_2.wav
8_lucas_49.wav
0_theo_20.wav
4_nicolas_11.wav
2_lucas_8.wav
8_jackson_35.wav
9_nicolas_22.wav
6_george_41.wav
0_jackson_26.wav
3_george_25.wav
0_yweweler_43.wav
4_yweweler_46.wav
1_lucas_41.wav
3_yweweler_39.wav
0_theo_37.wav
1_lucas_44.wav
1_george_22.wav
8_george_12.wav
3_nicolas_35.wav
1_george_6.wav
3_nicolas_18.wav
9_theo_22.wav
7_jackson_6.wav
4_theo_36.wav
7_george_40.wav
5_nicolas_5.wav
9_lucas_11.wav
2_george_9.wav
6_jackson_36.wav
4_lucas_28.wav
4_nicolas_46.wav
0_nicolas_13.wav
0_nicolas_49.wav
1_yweweler_21.wav
1_george_33.wav
2_lucas_7.wav
5_theo_20.wav
0_george_49.wav
9_nicolas_47.wav
4_george_25.wav
1_yweweler_23.wav
7_nicolas_15.wav
1_lucas_29.wav
9_yweweler_22.wav
1_nicolas_15.wav
0_theo_46.wav
6_

In [ ]:
from audio_classification_features import audio_features as af

In [ ]:
af().extractor('cl_dataset')

Class : Label
0 : 0
1 : 1
2 : 2
3 : 3
4 : 4
5 : 5
6 : 6
7 : 7
8 : 8
9 : 9

Building Features...


100%|██████████| 20148/20148 [00:38<00:00, 520.32it/s]


In [ ]:
import pandas as pd
import os
import shutil
import numpy as np
import pickle
from tqdm import tqdm
import keras
from scipy.io import wavfile
from python_speech_features import mfcc
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Dropout,Flatten
import librosa
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical

In [ ]:
def get_conv_model():
  model=Sequential()
  model.add(Conv2D(16, (3,3), activation='relu', strides=(1,1),
                   padding='same',input_shape=(9,13,1)))
  model.add(MaxPooling2D(2,2))
  model.add(Conv2D(32, (3,3), activation='relu', strides=(1,1),
                   padding='same'))
  model.add(MaxPooling2D(2,2))
  model.add(Conv2D(64, (3,3), activation='relu', strides=(1,1),
                   padding='same'))
  model.add(MaxPooling2D(2,2))
  model.add(Dropout(0.5))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(10, activation='softmax'))
  model.summary()
  model.compile(optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=1e-8, amsgrad=False),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  return model

In [ ]:
model=get_conv_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 9, 13, 16)         160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 4, 6, 16)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 6, 32)          4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 2, 3, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 2, 3, 64)          18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 1, 64)          0         
_________________________________________________________________
dropout (Dropout)            (None, 1, 1, 64)          0

In [ ]:
best_model_file = "best.h5"
best_model = ModelCheckpoint(best_model_file, monitor='val_accuracy', verbose=1, save_best_only=True)

In [ ]:
import numpy
x=np.load('x.npy')
y=np.load('y.npy')

In [ ]:
results = model.fit(x,y,epochs=10, batch_size=32,
          shuffle=True, validation_split=0.2, verbose=1,callbacks=[best_model])

Epoch 1/10
504/504 [==============================] - ETA: 0s - loss: 1.9738 - accuracy: 0.2407
Epoch 00001: val_accuracy improved from -inf to 0.38660, saving model to best.h5
504/504 [==============================] - 2s 4ms/step - loss: 1.9738 - accuracy: 0.2407 - val_loss: 1.6114 - val_accuracy: 0.3866
Epoch 2/10
495/504 [============================>.] - ETA: 0s - loss: 1.5824 - accuracy: 0.3949
Epoch 00002: val_accuracy improved from 0.38660 to 0.48685, saving model to best.h5
504/504 [==============================] - 2s 4ms/step - loss: 1.5816 - accuracy: 0.3952 - val_loss: 1.3931 - val_accuracy: 0.4868
Epoch 3/10
498/504 [============================>.] - ETA: 0s - loss: 1.4134 - accuracy: 0.4559
Epoch 00003: val_accuracy improved from 0.48685 to 0.54888, saving model to best.h5
504/504 [==============================] - 2s 4ms/step - loss: 1.4138 - accuracy: 0.4557 - val_loss: 1.2021 - val_accuracy: 0.5489
Epoch 4/10
489/504 [============================>.] - ETA: 0s - loss: 

In [ ]:
from keras.models import load_model

In [ ]:
model=load_model('best.h5')

In [ ]:
tp=0
fp=0
for r,d,f in os.walk('test/'):
  for files in f:
    c=af().make_prediction('test/'+files,10,model)
    print("GOt: {}, Was: {}".format(c,files[0]))
    if c==int(files[0]):
      tp+=1
    else:
      fp+=1

Extracting audio features and making prediction
GOt: 7, Was: 7
Extracting audio features and making prediction
GOt: 4, Was: 4
Extracting audio features and making prediction
GOt: 1, Was: 1
Extracting audio features and making prediction
GOt: 8, Was: 6
Extracting audio features and making prediction
GOt: 7, Was: 7
Extracting audio features and making prediction
GOt: 0, Was: 0
Extracting audio features and making prediction
GOt: 8, Was: 8
Extracting audio features and making prediction
GOt: 2, Was: 2
Extracting audio features and making prediction
GOt: 8, Was: 2
Extracting audio features and making prediction
GOt: 6, Was: 6
Extracting audio features and making prediction
GOt: 5, Was: 5
Extracting audio features and making prediction
GOt: 0, Was: 0
Extracting audio features and making prediction
GOt: 1, Was: 1
Extracting audio features and making prediction
GOt: 1, Was: 1
Extracting audio features and making prediction
GOt: 2, Was: 2
Extracting audio features and making prediction
GOt: 5,

In [ ]:
print("Correct: {}".format(tp))
print("False: {}".format(fp))
print("Score: {}".format((tp/(tp+fp))*100))

Correct: 624
False: 69
Score: 90.04329004329004
